<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Evaluator_ChromaDB_Post_Trainining_synthetic_text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Absolutely! Here's a concise summary of the code's functionality:

1. **Setup:**
   - Initializes a ChromaDB client and sets up a collection to store SQL queries and their embeddings.
   - Loads a fine-tuned PEFT Mistral model designed for text-to-SQL generation.
   - Loads your custom test dataset and preprocesses it.

2. **Embedding Generation:**
   - Defines a function `generate_embedding` to generate embeddings and SQL queries for input questions.
   - Utilizes the loaded model to generate SQL queries from your questions.
   - Creates embeddings (numerical representations) for the generated SQL queries to capture their semantic meaning.
   - Stores both the embeddings and their corresponding SQL queries and original answers in the ChromaDB collection for future retrieval.

3. **Querying and Evaluation:**
   - For each question in your test dataset:
      - Generates an embedding for the question.
      - Queries the ChromaDB collection to find the top 5 most semantically similar SQL queries based on the embedding.
      - Prints the original query, the generated SQL query, and the top 5 most similar queries retrieved from ChromaDB, along with their original answers.

**Key Improvements and Features:**

- **PEFT Model Optimization:** Uses Parameter-Efficient Fine-Tuning (PEFT) to reduce the model's memory footprint and improve inference speed.
- **Batch Processing:** Processes input data in batches for more efficient embedding generation.
- **Parallel Embedding Generation:** Uses `ThreadPoolExecutor` to parallelize embedding generation on multiple CPU threads.
- **Pre-Tokenization:** Pre-tokenizes questions for faster embedding generation.
- **ChromaDB Storage:** Leverages ChromaDB's efficient vector storage and search capabilities to store and retrieve SQL queries based on semantic similarity.
- **Error Handling:** Includes `try-except` blocks and logging to handle errors gracefully and provide informative messages.
- **Customizable:**  You can easily adjust the `num_samples_to_process` and `top_k_results` parameters to control how much data is processed and how many similar queries are retrieved.
- **Progress Bar:**  Uses `tqdm` to display a progress bar during embedding generation, providing visual feedback on the process.

**Overall:**

This code provides a framework for evaluating your fine-tuned text-to-SQL model by generating embeddings, storing them in ChromaDB, and then retrieving and evaluating the most similar SQL queries for new questions. This approach enables you to assess the model's ability to understand the intent behind natural language questions and generate semantically relevant SQL queries, even if they are not exact matches to the original.


In [ ]:
!pip install -q datasets
!pip install -q chromadb
!pip install -q faiss-gpu
!pip install peft  -q

!pip install bitsandbytes -q
!pip pip install accelerate -q

!pip install -U flash-attn --no-build-isolation --quiet

!pip install colab-env --quiet

!pip install mistral_inference -q

!pip install -q evaluate sentence_transformers

In [ ]:
!nvidia-smi

Sun Jul  7 04:56:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   41C    P8              16W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
import colab_env
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

# Environment Settings

In [ ]:
import logging
from tqdm.auto import tqdm
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
import os


from sentence_transformers import SentenceTransformer
import chromadb

# Logging Setup
logging.basicConfig(level=logging.INFO)

# 1. Configurable Parameters
PEFT_MODEL_ID = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2-dataeval"

#b-mc2/sql-create-context
#DATASET_FILE = "/content/drive/MyDrive/datasets/test_dataset.json"

#gretelai/synthetic_text_to_sql
DATASET_FILE = "/content/drive/MyDrive/datasets/gretelai_test_dataset.json"


NUM_SAMPLES_TO_PROCESS = int(os.getenv("NUM_SAMPLES", 10))
GENERATION_PARAMS = {
    "max_new_tokens": 256, "do_sample": True, "temperature": 0.7, "top_k": 50, "top_p": 0.95
}
SIMILARITY_THRESHOLD = 0.75

# 2. Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

# 3. Load Evaluation Dataset
eval_dataset = load_dataset("json", data_files=DATASET_FILE, split="train")
if NUM_SAMPLES_TO_PROCESS > 0:
    eval_dataset = eval_dataset.select(range(NUM_SAMPLES_TO_PROCESS))
logging.info(f"Processing {len(eval_dataset)} samples from the dataset.")


# 4. Load Models and Tokenizer
logging.info(f"Loading fine-tuned PEFT model from: {PEFT_MODEL_ID}")
model = AutoPeftModelForCausalLM.from_pretrained(PEFT_MODEL_ID)
tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL_ID)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, **GENERATION_PARAMS)
logging.info("Model and tokenizer loaded successfully!")

# 5. ChromaDB Setup
client = chromadb.PersistentClient(path='db')  # Store embeddings on disk
collection = client.get_or_create_collection(name="sql_queries_and_embeddings")

# Add Original SQL Queries to ChromaDB
embedding_model = SentenceTransformer("all-mpnet-base-v2")
original_sql_queries = [
    item['messages'][2]['content']
    for item in eval_dataset if len(item['messages']) > 2 and item['messages'][2].get('content')
]

sql_embeddings = embedding_model.encode(original_sql_queries).tolist()
collection.add(
    embeddings=sql_embeddings,
    metadatas=[{"original_sql": query} for query in original_sql_queries],
    ids=[f"original_{i}" for i in range(len(original_sql_queries))]  # Unique IDs
)


# Postgresql Setup

In [ ]:
#ADDED By FM 01/06/2024
!apt-get update -y
!apt-get install postgresql-14 -y

!service postgresql restart
!sudo apt install postgresql-server-dev-all

In [ ]:
# PostGRES SQL Settings
!sudo -u postgres psql -c "CREATE USER postgres WITH SUPERUSER"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

ERROR:  role "postgres" already exists
ALTER ROLE


In [ ]:
QUERY_create='CREATE TABLE table_name_24 (score VARCHAR, date VARCHAR)'

In [ ]:
QUERY_select='SELECT 2009 FROM table_name_50 WHERE 2011 = "a"'

In [ ]:
def table_creator(query):
    import os
    import psycopg2 as ps
    import pandas as pd

    DB_NAME = "postgres"
    DB_USER = "postgres"
    DB_PASS = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5432"

    conn = ps.connect(database=DB_NAME,
                  user=DB_USER,
                  password=DB_PASS,
                  host=DB_HOST,
                  port=DB_PORT)

    cur = conn.cursor() # creating a cursor




    # Wrap the execute command in a try-except block to handle potential errors
    try:
        cur.execute("""
                            %s
                            """%query)
        conn.commit()
        print("Table Created successfully")
    except Exception as e:
        conn.rollback() # Rollback the transaction in case of an error
        print("Error creating table:", e)

    conn.close()

In [ ]:
import os
import psycopg2 as ps
import pandas as pd

DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASS = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

In [ ]:
import os
import psycopg2 as ps
import pandas as pd

def table_select(query):
    conn = ps.connect(database=DB_NAME,
                      user=DB_USER,
                      password=DB_PASS,
                      host=DB_HOST,
                      port=DB_PORT)
    #print("Database connected successfully")

    query = query.replace('"', "'") # Replace double quotes with single quotes for potential date values

    try:
        #df = pd.read_sql_query("%s"%query, con=conn)
        print('rec: %'%df) # Print the resulting DataFrame
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()

        for row in results:
            print(row)

            print()

            # Commit the transaction to save the changes
            conn.commit()
            #print("QUERY successfully")
            print()

            # Close the cursor and connection
            cursor.close()
            conn.close()
    except Exception as e:
        #conn.rollback() # Rollback the transaction in case of an error
        #print("Error executing query:", e)
        print('TABLE IS EMPTY')


        conn.close()
    #return bad

In [ ]:
table_creator(QUERY_create)

Table Created successfully


# Model Evaluator

In [ ]:
# 6. Evaluation Function (Exact Match Only)
def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()

    #print("\n\n")
    question = sample["messages"][1]["content"]
    original_answer = sample["messages"][2]["content"]


    schema=sample["messages"][0]['content']
    schema_query=schema[153:len(schema)]

    print(f'Question: {question}')
    print(f'SCHEMA: {schema_query}')
    print(f'Original Answer: {original_answer}')
    print(f'Generated Answer: {predicted_answer}')

    if predicted_answer == original_answer:

        print("\n")
        print(f'SCHEMA QUERY: {schema_query}')
        table_creator(schema_query)
        print("\n")
        print(f'Generated Answer: {predicted_answer}')
        table_select(predicted_answer)
        print("\n")
        print('MATCH')
        return 1

    # If not an exact match, check semantic similarity using ChromaDB:
    predicted_embedding = embedding_model.encode([predicted_answer]).tolist()[0]
    results = collection.query(
        query_embeddings=[predicted_embedding],
        n_results=1,
        include=["distances", "metadatas"]
    )
    closest_distance = results['distances'][0][0]
    most_similar_query = results['metadatas'][0][0]['original_sql']
    print(f'Closest Distance: {closest_distance}')

    similarity_threshold = SIMILARITY_THRESHOLD

    if closest_distance < similarity_threshold:
        print("\n")
        print('MATCH (Semantically Similar)')
        print("\n")
        print(f'SCHEMA QUERY: {schema_query}')
        table_creator(schema_query)
        print("\n")
        print('Similar Query:', most_similar_query)
        table_select(most_similar_query)
        print("\n")
        return 1

    else:
        print('NO MATCH')
        return 0

    print("\n\n")

# 7. Main Evaluation Loop
success_rate = []
for i, s in enumerate(tqdm(eval_dataset)):
    print()
    print(f"Evaluating sample: {i}")
    try:
        success_rate.append(evaluate(s))
    except Exception as e:
        logging.error(f"Error evaluating sample {i}: {e}")


# 8. Compute and Print Accuracy
if len(success_rate) > 0:
    accuracy = sum(success_rate) / len(success_rate)
    print(f"\nAccuracy: {accuracy:.2%}\n")
else:
    print("\nNo samples were successfully evaluated. Check the dataset and evaluation logic.\n")

  0%|          | 0/10 [00:00<?, ?it/s]


Evaluating sample: 0
Question: Show ids for all documents in type CV without expense budgets.
SCHEMA: CREATE TABLE Documents_with_expenses (document_id VARCHAR, document_type_code VARCHAR); CREATE TABLE Documents (document_id VARCHAR, document_type_code VARCHAR)
Original Answer: SELECT document_id FROM Documents WHERE document_type_code = "CV" EXCEPT SELECT document_id FROM Documents_with_expenses
Generated Answer: SELECT document_id FROM Documents WHERE document_type_code <> 'CV' EXCEPT SELECT document_id FROM Documents_with_expenses
Closest Distance: 0.009356578506243534


MATCH (Semantically Similar)


SCHEMA QUERY: CREATE TABLE Documents_with_expenses (document_id VARCHAR, document_type_code VARCHAR); CREATE TABLE Documents (document_id VARCHAR, document_type_code VARCHAR)
Table Created successfully


Similar Query: SELECT document_id FROM Documents WHERE document_type_code = "CV" EXCEPT SELECT document_id FROM Documents_with_expenses
TABLE IS EMPTY



Evaluating sample: 1
Questio